In [1]:
#!/usr/bin/env python

import os
import glob
import argparse 
import subprocess as sub
import shlex 
from subprocess import PIPE, Popen
import pandas as pd
import logging
import re
import datetime
import sys


In [2]:
def getting_bnum_tnum_list(bnum_tnum_csv): 
    ########### Get bnum_tnum list: 
    savedir = os.getcwd()
    os.chdir(csv_dir)
    bnum_tnum_df = pd.read_csv(bnum_tnum_csv, header = None)
    bnum_tnum_df.columns = ['bnum', 'tnum', 'DUMMY']
    os.chdir(savedir)
    return bnum_tnum_df

In [11]:
def get_anat_data(measure = "median"):
    os.chdir(recgli_path_root+"/"+bnum+"/"+tnum)
    anat_data_file = pd.read_csv(glob.glob('svk_roi_analysis/*fse.csv')[0])
    if "roi_analysis" in os.listdir():
        os.chdir('roi_analysis')
        vial_files = glob.glob('*_t1ca_*.idf')
    if len(vial_files)>0: 
        ## shorten vial names
        vial_names = [vial.replace(".", "_").split("_")[2] for vial in vial_files]
        ## shorten the anatomic data down to only the biopsies of interest: 
        anat_data_biopsy_file = anat_data_file[anat_data_file['roi-label'].isin(vial_names)]
        ## find only the measure of interest: 
        anat_data_biopsy_file_measure = anat_data_biopsy_file[anat_data_biopsy_file['measure']==measure]
    else: 
        vial_names = []
        print("ERROR! No biopsies.")
    return anat_data_biopsy_file_measure

In [12]:
recgli_path_root = "/data/RECglioma"
bnum = "b1338"
tnum = "t8773"

In [13]:
os.chdir(recgli_path_root+"/"+bnum+"/"+tnum)

In [21]:
if "roi_analysis" in os.listdir():
    os.chdir('roi_analysis')
    vial_files = glob.glob('*_t1ca_*.idf')

In [22]:
vial_files

['t8773_t1ca_63B60.idf', 't8773_t1ca_61B60.idf', 't8773_t1ca_62B60.idf']

In [24]:
vial_names = [vial.replace(".", "_").split("_")[2] for vial in vial_files]
vial_names

['63B60', '61B60', '62B60']

In [25]:
anat_data_file

,tab-file,t-num,measure,roi-label,vol(cc),fse,fl,t1c,nfse,nfl,nt1c
0,t8773_roi_flt1cfse.tab,t8773,median,nbrain,1355.10,645.00,1295.00,2130.00,1.15,0.93,1.00
1,t8773_roi_flt1cfse.tab,t8773,median,brain,1378.79,645.00,1305.00,2130.00,1.15,0.94,1.00
2,t8773_roi_flt1cfse.tab,t8773,median,t2all,23.70,895.00,2415.00,2010.00,1.60,1.74,0.94
3,t8773_roi_flt1cfse.tab,t8773,median,cel,5.08,855.00,2525.00,3910.00,1.53,1.82,1.83
4,t8773_roi_flt1cfse.tab,t8773,median,nec,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,t8773_roi_flt1cfse.tab,t8773,median,nel,18.61,915.00,2365.00,1870.00,1.63,1.70,0.87
6,t8773_roi_flt1cfse.tab,t8773,median,t1c12c,5.06,855.00,2525.00,3930.00,1.53,1.82,1.84
7,t8773_roi_flt1cfse.tab,t8773,median,t1c12,6.08,845.00,2525.00,3750.00,1.51,1.82,1.75
8,t8773_roi_flt1cfse.tab,t8773,median,t1c08c,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,t8773_roi_flt1cfse.tab,t8773,median,t1c08,6.04,1155.00,2535.00,1410.00,2.06,1.82,0.66


,tab-file,t-num,measure,roi-label,vol(cc),fse,fl,t1c,nfse,nfl,nt1c


In [ ]:
anat_data_biopsy_file_measure = anat_data_biopsy_file[anat_data_biopsy_file['measure']==measure]

In [ ]:
anat_data_biopsy_file_measure

In [ ]:

parser = argparse.ArgumentParser(description='Create a systematic logger of whether biopsies are valid or not through visual inspection')
parser.add_argument("--csv_name",        required=True,    help='Precise name of the csv file that contains the perfusion files of interest.')
parser.add_argument("--csv_dir",         required=True,    help='Precise path of the csv file that contains the perfusion files of interest.')
parser.add_argument("--cohort_name",     required=True,    help='Precise cohort name of the scans of interest (e.g. "po1_preop_recur" or "REC_HGG")')
parser.add_argument("--output_file",     required=True,    help="Name of the output file csv")
parser.add_argument("--output_dir",      required=True,    help="Path where output files get written ")
parser.add_argument("--measure",         required=False,   help='Measurement that you desire for the anatomical imaging (e.g. median, mean, max, etc.)', default = "median")
parser.add_argument("-v", "--verbose",                     help = "verbose output", action='store_true', default=False,   required=False)


In [ ]:
args = parser.parse_args("--csv_dir /home/sf673542/DataWrangling/GetMergeData/Oct2018/ --csv_name ajnr_bnum_tnum.95.csv --cohort_name REC_HGG --output_file REC_HGG_anat_biopsies.csv --output_dir /home/sf673542/DataWrangling/GetMergeData/Oct2018/ --measure median".split())

In [ ]:

#####################################
#   Create strings of the arguments for 
#   navigating to correct directory
#####################################
cohort_name     = args.cohort_name
csv_name        = args.csv_name
csv_dir         = args.csv_dir
output_file     = args.output_file
output_dir      = args.output_dir
measure         = args.measure

print("===============================================")
print("scan list dir:     ", csv_dir)
print("scan list name:    ", csv_name)
print("cohort name:       ", cohort_name) 
print("output file name:  ", output_file) 
print("output dir:        ", output_dir)
print("measurement:       ", measure)
print("===============================================")


In [ ]:

#####################################
#   Reading in the csv_name
#   as the scan listls
#####################################
bnum_tnum_df = getting_bnum_tnum_list(csv_name)
bnum_tnum_df

In [ ]:
#####################################
#   Setting the roots of the data 
#   based on the cohort name
#####################################

if cohort_name == 'po1_preop_recur': 
    recgli_path_root = '/data/RECglioma/archived/'
elif cohort_name == 'REC_HGG': 
    recgli_path_root = '/data/RECglioma/'
else: 
    print('Please use a valid cohort name, REC_HGG or po1_preop_recur.')
    exit(1)

In [ ]:
recgli_path_root

In [ ]:
#####################################
#   Instantiate the total anatomical dataframe
#####################################

anat_total_df = pd.DataFrame()


#####################################
#   Iterating through scans, grabbing the data of interest: 
#####################################

for index, row in bnum_tnum_df.iterrows():
    bnum = row['bnum']
    tnum = row['tnum']

    print(bnum)
    print(tnum)

    #####################################
    #   Change into the correct directory
    #####################################
    os.chdir(recgli_path_root+"/"+bnum+"/"+tnum)

    #####################################
    #   Gather the vialIDs 
    #####################################
    if "roi_analysis" in os.listdir(): 
        os.chdir('roi_analysis')
        vial_files = glob.glob('*_t1ca_*.idf')
    else: 
        vial_files = []
        print('ERROR! No biopsies.')

    #####################################
    #   Change back into the correct directory
    #####################################
    os.chdir(recgli_path_root+"/"+bnum+"/"+tnum)

    #####################################
    #   Gather the data if available: 
    #####################################

    ## If there exists an anatomical data file: 
    if len(glob.glob('svk_roi_analysis/*fse.csv'))>0: 
        print('Anatomical csv file found')         
        ## If there exists biopsy masks in roi_analysis: 
        if len(vial_files)>0: 
            print('vialIDs found in roi_analysis')
            ## Set the vial_file_status = 1 so that we know they exist 
            vial_files_status = 1

            ## Instanstiate the tnum_anatomical_df
            tnum_anatomical_df  = pd.DataFrame()

            ## Get the data: 
            tnum_anatomical_df = get_anat_data()

            ## Add these data to the overall dataFrame: 
            anat_total_df = anat_total_df.append(tnum_anatomical_df, ignore_index= True)\

        else: 
            print('No vial IDs found! No biopsies.')

    else: 
        print("------------------------------------------")
        print('No fse.csv file! run svk_roi_analysis')
        print('------------------------------------------')
        
    

In [ ]:
anat_total_df

In [ ]:
cols = ['t-num', 'roi-label', 'fse', 'fl', 't1c', 't1v', 't1d', 'nfse', 'nfl', 'nt1c', 'nt1v', 'nt1d']

In [ ]:
anat_total_df = anat_total_df[cols]

In [ ]:
anat_total_df.columns = ['tnum', 'roi.label', 'fse', 'fl', 't1c', 't1v', 't1d', 'nfse', 'nfl', 'nt1c', 'nt1v', 'nt1d']

In [ ]:
anat_total_df

In [ ]:
out_file = output_dir+output_file

In [ ]:
anat_total_df.to_csv(out_file, index=False)